In [ ]:
### [Flags]

# Processing type of person's name (Choose one of them)
flag_nlp_name = 'ginza_electra'
#flag_nlp_name = 'ginza'
#flag_nlp_name = 'none'


In [ ]:
### [Setup]

# Install necessary libraries on Google Colab
# check if running on Google Colab
if 'google.colab' in str(get_ipython()):

    # install Selenium
    !pip -q install selenium

    # install Chromium
    !apt -qq update
    !apt -qq install chromium-chromedriver

    # install GiNZA
    if flag_nlp_name == 'ginza':
        !pip -qq install ja_ginza
    elif flag_nlp_name == 'ginza_electra':
        !pip -qq install -U ginza https://github.com/megagonlabs/ginza/releases/download/latest/ja_ginza_electra-latest-with-model.tar.gz


In [ ]:
import sys
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import spacy
import regex
from spacy import displacy

### ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ----------
### [Settings]

last_article = ""

exclude_list = ["(word1)", "(word2)"]

num_page = 10

url = "https://sports.yahoo.co.jp/list/news/ws?genre=ws"
### ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ----------

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# タイトル解析用
kanji_pattern = regex.compile(r'\p{Script=Han}+')

if flag_nlp_name == 'ginza':
    nlp = spacy.load('ja_ginza')
elif flag_nlp_name == 'ginza_electra':
    nlp = spacy.load('ja_ginza_electra')

# ブラウザのオプションを格納する変数を取得
options = webdriver.ChromeOptions()

# Headlessモードを有効に（コメントアウトするとブラウザが実際に立ち上がる）
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ブラウザを起動
driver = webdriver.Chrome(options=options)

# ページ取得
driver.get(url)

# 待機設定
driver.implicitly_wait(10)

# 「もっと見る」ボタンをクリック
i = 0
while i < num_page:
    driver.find_element(By.ID, "moreViewButton").click()
    i += 1

html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html5lib")

# ページを表示
#print(soup.prettify())

body = soup.select('body')

num_extracted = 0
num_total = 0
num_exc_list = 0
num_exc_name = 0
flag = False
latest_article = ""
for article in body[0].find_all(class_="cm-timeLine__item"):
    num_total += 1
    flag = False
    title = article.find(class_="cm-timeLine__itemTitle").get_text()
    link = article.find(class_="cm-timeLine__itemArticleLink").get("href")

    if latest_article == "":
        latest_article = "last_article = \"" + link + "\""
        
    if link == last_article:
        print("---------- (Cut) ----------")
        break

    for e in exclude_list:
        if e in title:
            flag = True
            num_exc_list += 1

    # タイトル解析(人名を抽出)
    if flag_nlp_name in ['ginza', 'ginza_electra']:
        doc = nlp(title)
        for ent in doc.ents:
            if ent.label_ == 'Person':

                # 人名に漢字が含まれている場合、除外する
                if kanji_pattern.search(ent.text) != None:
                    flag = True
                    num_exc_name += 1

    if flag == False:
        #displacy.render(doc, style="ent", options={"compact":True}, jupyter=True)
        num_extracted += 1
        print(title, "\n", link,"\n\n")

print(f'# of total articles:     {num_total}')
print(f'# of extracted articles: {num_extracted}')
print(f'# of skipped articles:   {num_total - num_extracted}')
print(f' (exlude_list):          {num_exc_list}')
print(f' (person\'s name):        {num_exc_name}\n')

print(latest_article, "\n\n")
